## Коллокации

Коллокации - это устойчивые выражения, состоящие из двух и более слов. Устойчивые - значит, что они часто используются вместе. Также часто значения коллокации не могут быть выведены лишь из значений, входящих в них слов.

In [15]:
from nltk import sent_tokenize, word_tokenize

In [3]:
import itertools
from razdel import sentenize
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
import re
from string import punctuation
from nltk.corpus import stopwords
import json
from math import log

stops = set(stopwords.words('russian') + ["это", "весь"])
morph = MorphAnalyzer()

def normalize(text):
    tokens = re.findall('[а-яёa-z0-9]+', text.lower())
    normalized_text = [morph.parse(word)[0].normal_form for word in tokens]
    normalized_text = [word for word in normalized_text if len(word) > 2 and word not in stops]
    
    return normalized_text

def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, stops, n=2):
    ngrams = []
    tokens = [token for token in tokens if token not in stops]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

Предобработаем почти также, только теперь нам не нужны тэги начала и конца.

In [4]:
fn = '../data/preprocessed_corpus.json'
# corpus = open('lenta.txt').read()
# corpus = preprocess(corpus)

# with open(fn, 'w') as f:
#     json.dump(corpus, f)
with open(fn) as f:
    corpus = json.load(f)

In [5]:
word_counter = Counter()

for sent in corpus:
    word_counter.update(ngrammer(sent, stops=[stops]))

In [6]:
len(word_counter)

589668

In [7]:
word_counter.most_common(15)

[(('риа', 'новость'), 3508),
 (('сообщать', 'риа'), 1341),
 (('владимир', 'путин'), 1045),
 (('миллион', 'доллар'), 972),
 (('итар', 'тасс'), 884),
 (('2000', 'год'), 870),
 (('1999', 'год'), 781),
 (('настоящее', 'время'), 771),
 (('пресс', 'служба'), 751),
 (('президент', 'россия'), 743),
 (('федеральный', 'сила'), 700),
 (('правоохранительный', 'орган'), 631),
 (('премьер', 'министр'), 602),
 (('северный', 'кавказ'), 595),
 (('уголовный', 'дело'), 576)]

В списке есть коллокации, которые попали в список из-за того, что одно слово очень частотное и вообще встречается много в каких контекстах. Нас скорее интересуют случаи, когда слова в большинстве случаев встречаются вместе. Для этого мы можем придумать какие-нибудь формулы, учитывающие частоты слов по отдельности и общую частоту.

Самый простой способ - взять количество упоминаний биграма и поделить на сумму количеств упоминаний слов по отдельности. 

Такая формула называется PMI (pointwise mutual information).

In [70]:
def scorer_simple(word_count_a, word_count_b, bigram_count, *args):
    try:
        score = log(bigram_count/(word_count_a+word_count_b))
    
    except ZeroDivisionError:
        return 0
    
    return score

def scorer_simple2(word_count_a, word_count_b, bigram_count, *args):
    try:
        score = log(bigram_count/(word_count_a*word_count_b))
    
    except ZeroDivisionError:
        return 0
    
    return score

Сделаем функцию, которая будет делать счетчики для слов и биграммов.

In [45]:
def collect_stats(corpus, stops):
    ## соберем статистики для отдельных слов
    ## и биграммов
    
    unigrams = Counter()
    bigrams = Counter()
    
    for sent in corpus:
        unigrams.update(sent)
        bigrams.update(ngrammer(sent, stops, 2))
    
    return unigrams, bigrams

И функцию, которая пройдет по всем биграммам и вычислит для них нашу метрику.

In [75]:
def score_bigrams(unigrams, bigrams, scorer, 
                  threshold=-100000, min_count=1):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    len_vocab = len(unigrams)
    
    for bigram in bigrams:
        bigram_count = bigrams[bigram]

        score = scorer(unigrams[bigram[0]], unigrams[bigram[1]], 
                       bigram_count, 
                       len_vocab, min_count)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [11]:
unigrams, bigrams = collect_stats(corpus, stops)

In [76]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_simple)
bigram2score2 = score_bigrams(unigrams, bigrams, scorer_simple2)

In [48]:
# bigram2score.items()

Проблема с таким подходом в том, что на самом верху окажутся слова, которые встречают по одному разу.

In [77]:
sorted(bigram2score.items(), key=lambda x:x[1], reverse=True)

[(('сопоцкина', 'друскеник'), -0.6931471805599453),
 (('das', 'ist'), -0.6931471805599453),
 (('ist', 'nesteroff'), -0.6931471805599453),
 (('равнина', 'обманчивый'), -0.6931471805599453),
 (('трусливо', 'замешаться'), -0.6931471805599453),
 (('золочёный', 'воротамкенсингтонский'), -0.6931471805599453),
 (('чугунный', 'завиток'), -0.6931471805599453),
 (('конфета', 'плюшевый'), -0.6931471805599453),
 (('кенсингтонский', 'паркай'), -0.6931471805599453),
 (('балморал', 'будутприсутствовать'), -0.6931471805599453),
 (('соберутсяотдельный', 'олторп'), -0.6931471805599453),
 (('анди', 'хара'), -0.6931471805599453),
 (('каныбек', 'иманалий'), -0.6931471805599453),
 (('heaven', 'gate'), -0.6931471805599453),
 (('буэнос', 'айрес'), -0.6931471805599453),
 (('тагилонить', 'вгальский'), -0.6931471805599453),
 (('аршба', 'взасада'), -0.6931471805599453),
 (('cibc', 'oppenheimer'), -0.6931471805599453),
 (('губден', 'карабудахкентский'), -0.6931471805599453),
 (('хазбулатахазбулатов', 'магомедхабиб

In [50]:
sorted(bigram2score2.items(), key=lambda x:x[1], reverse=True)

[(('сопоцкина', 'друскеник'), 0.0),
 (('das', 'ist'), 0.0),
 (('ist', 'nesteroff'), 0.0),
 (('равнина', 'обманчивый'), 0.0),
 (('трусливо', 'замешаться'), 0.0),
 (('золочёный', 'воротамкенсингтонский'), 0.0),
 (('чугунный', 'завиток'), 0.0),
 (('конфета', 'плюшевый'), 0.0),
 (('кенсингтонский', 'паркай'), 0.0),
 (('балморал', 'будутприсутствовать'), 0.0),
 (('соберутсяотдельный', 'олторп'), 0.0),
 (('анди', 'хара'), 0.0),
 (('heaven', 'gate'), 0.0),
 (('тагилонить', 'вгальский'), 0.0),
 (('аршба', 'взасада'), 0.0),
 (('cibc', 'oppenheimer'), 0.0),
 (('губден', 'карабудахкентский'), 0.0),
 (('хазбулатахазбулатов', 'магомедхабиб'), 0.0),
 (('полуторагодовой', 'программыкредитование'), 0.0),
 (('уильсон', 'мейриали'), 0.0),
 (('вирусолог', 'элвин'), 0.0),
 (('ноер', 'муис'), 0.0),
 (('manneken', 'pis'), 0.0),
 (('еговерный', 'гардеробщик'), 0.0),
 (('франсис', 'сейерстед'), 0.0),
 (('сейерстед', 'francis'), 0.0),
 (('francis', 'sejerstad'), 0.0),
 (('daniel', 'devaud'), 0.0),
 (('devaud',

In [51]:
bigram2score.most_common()

[(('сопоцкина', 'друскеник'), -0.6931471805599453),
 (('das', 'ist'), -0.6931471805599453),
 (('ist', 'nesteroff'), -0.6931471805599453),
 (('равнина', 'обманчивый'), -0.6931471805599453),
 (('трусливо', 'замешаться'), -0.6931471805599453),
 (('золочёный', 'воротамкенсингтонский'), -0.6931471805599453),
 (('чугунный', 'завиток'), -0.6931471805599453),
 (('конфета', 'плюшевый'), -0.6931471805599453),
 (('кенсингтонский', 'паркай'), -0.6931471805599453),
 (('балморал', 'будутприсутствовать'), -0.6931471805599453),
 (('соберутсяотдельный', 'олторп'), -0.6931471805599453),
 (('анди', 'хара'), -0.6931471805599453),
 (('каныбек', 'иманалий'), -0.6931471805599453),
 (('heaven', 'gate'), -0.6931471805599453),
 (('буэнос', 'айрес'), -0.6931471805599453),
 (('тагилонить', 'вгальский'), -0.6931471805599453),
 (('аршба', 'взасада'), -0.6931471805599453),
 (('cibc', 'oppenheimer'), -0.6931471805599453),
 (('губден', 'карабудахкентский'), -0.6931471805599453),
 (('хазбулатахазбулатов', 'магомедхабиб

Поэтому можно немного переделать оценивающую функцию, добавив минимальное число вхождений для биграмма (на этот раз без log).

In [106]:
def scorer(word_count_a, word_count_b, bigram_count, len_vocab, min_count):
    try:
        score = (bigram_count - min_count) / ((word_count_a + word_count_b))
    except ZeroDivisionError:
        return 0
    
    return score

In [105]:
log(-1)

ValueError: math domain error

In [107]:
bigram2score = score_bigrams(unigrams, bigrams, scorer, min_count=20)

In [108]:
bigram2score.most_common(15)

[(('риа', 'новость'), 0.4664348756351966),
 (('итар', 'тасс'), 0.4549763033175355),
 (('associated', 'press'), 0.43983402489626555),
 (('new', 'york'), 0.4262295081967213),
 (('сей', 'пора'), 0.3801955990220049),
 (('взрывной', 'устройство'), 0.3578088578088578),
 (('ножать', 'юртовский'), 0.3522727272727273),
 (('рао', 'еэс'), 0.3514056224899598),
 (('полевой', 'командир'), 0.34609720176730485),
 (('санкт', 'петербург'), 0.3456998313659359),
 (('исполнять', 'обязанность'), 0.32946298984034833),
 (('северный', 'кавказ'), 0.32596371882086167),
 (('карачаево', 'черкесия'), 0.32571428571428573),
 (('правоохранительный', 'орган'), 0.3167444271643339),
 (('населить', 'пункт'), 0.31583552055993)]

В статье про Word2Vec для создания нграммов использовалась такая функция:

In [99]:
def scorer_w2v(word_count_a, word_count_b, bigram_count, len_vocab, min_count):

    try:
        score = ((bigram_count - min_count) * len_vocab) / (word_count_a * word_count_b)
    except ZeroDivisionError:
        return 0
    
    return score

Посмотрим, отличается ли она от нашей.

In [102]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_w2v, min_count=15)

In [103]:
bigram2score.most_common(15)

[(('dow', 'jones'), 840.917487684729),
 (('exit', 'polls'), 823.0413105413105),
 (('рассесть', 'джонстон'), 750.3571428571429),
 (('сьерра', 'леон'), 744.8241633579127),
 (('брюшной', 'тиф'), 731.3063063063063),
 (('кабардино', 'балкария'), 726.6304347826087),
 (('норильский', 'никель'), 702.2058823529412),
 (('сбс', 'агро'), 696.6180371352785),
 (('wall', 'street'), 660.6918238993711),
 (('кофи', 'аннана'), 656.5625),
 (('кох', 'везер'), 656.5625),
 (('street', 'journal'), 602.5134952766532),
 (('подписка', 'невыезд'), 602.0652173913044),
 (('шри', 'ланка'), 577.1978021978022),
 (('элла', 'памфилов'), 551.2368815592204)]

Во всех случаях выше мы считали нграммами только слова, которые встречаются друг за другом. Но в нграммы часто можно ещё что-то вставить. Например, "принять участие" может превратиться в "принять самое активное/непосредственное участие". 

Чтобы отловить такие случаи можно считать нграммами слова, которые встречаются внутри какого-то окна. И считать по ним все те же метрики.

Можно ещё посчитать стандартное отклонение расстояния между двумя словами. Если оно маленькое - слова обычно стоят на строгой позиции по отношению друг к другу.

In [45]:
from collections import defaultdict
def get_window_stats(texts, window=8):
    
    bigrams = defaultdict(list)
    
    # проходим окном по текстам 
    # берем первое слово и считаем его целевым
    # проходим по остальным словам и их индексам
    # добавляем в словарь пары (целевое слов, текущее слово)
    # и добавляем индекс текущего в список этой пары
    # так мы получаем (слово_1,слово_2):[1,2,1,1,3,2]
    # порядок в этом случае учитывается - (слово_2, слово_1) - другая запись
    for text in texts:
        for i in range(len(text)-window):
            words = list(enumerate(text[i:i+window]))
            target = words[0][1]
            for j, word in words[1:]:
                bigrams[(target, word)].append(j)
    
    bigrams_stds = Counter()
    for bigram in bigrams:
        # выкидываем биграмы встретившиеся < 5 раз
        if len(bigrams[bigram]) > 5:
            bigrams_stds[bigram] = np.std(bigrams[bigram])
    
    return bigrams_stds

In [88]:
np.std([2, 4, 4, 4, 5, 5, 7, 9])

2.0

In [79]:
collocations = get_window_stats(corpus)

In [80]:
for col, score in collocations.items():
    print(col, score)
    break

('русский', 'год') 0.8329931278350429


In [58]:
sorted(cols.items(), key=lambda x:x[1], reverse=False)[:30]

[(('сентябрь', '1914'), 0.0),
 (('учебный', 'заведение'), 0.0),
 (('тихий', 'океан'), 0.0),
 (('охотный', 'ряд'), 0.0),
 (('террористический', 'акт'), 0.0),
 (('российско', 'японский'), 0.0),
 (('общий', 'объесть'), 0.0),
 (('должный', 'составить'), 0.0),
 (('данный', 'момент'), 0.0),
 (('совет', 'мвф'), 0.0),
 (('решаться', 'вопрос'), 0.0),
 (('нью', 'йоркский'), 0.0),
 (('государственный', 'гтк'), 0.0),
 (('комитет', 'гтк'), 0.0),
 (('сахалинский', 'область'), 0.0),
 (('южный', 'сахалинск'), 0.0),
 (('бывший', 'шеф'), 0.0),
 (('виктор', 'илюхин'), 0.0),
 (('основный', 'задача'), 0.0),
 (('подземный', 'толчок'), 0.0),
 (('северо', 'запад'), 0.0),
 (('ожесточённый', 'бой'), 0.0),
 (('карамах', 'чабанмах'), 0.0),
 (('министерство', 'оборона'), 0.0),
 (('сегодняшний', 'день'), 0.0),
 (('пропасть', 'вести'), 0.0),
 (('станислав', 'дерево'), 0.0),
 (('карачаево', 'черкесия'), 0.0),
 (('участник', 'митинг'), 0.0),
 (('тур', 'президентский'), 0.0)]

По этой ссылке можно прочитать про другие метрики.

http://www.scielo.org.mx/scielo.php?script=sci_arttext&pid=S1405-55462016000300327#t1

### Все готовое

Писать все это самому конечно не обязательно.

Удобно пользоваться phraser из gensim'а. Он собирает статистику по корпусу, а затем склеивает слова в биграммы.

In [81]:
import gensim

In [82]:
# собираем статистики
ph = gensim.models.Phrases(corpus)

In [83]:
# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)

По умолчанию там используется метрики из статьи про ворд2век и ещё есть нормализованные pmi.
Если не нравятся функции оценки, то ему можно подать любую другую функцию. Интерфейс у функции там почти точно такой же как и у наших.

In [84]:
# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)

In [316]:
p2[p[corpus[333]]]

['установить',
 'взрыв_произойти',
 'третье',
 'уровнечетвертый',
 'ярус',
 'комплекс',
 'зал',
 'игровой',
 'автомат']

Ну и наконец нграммы есть в нлтк. Тут больше метрик, но преборазователь слов в нграммы нужно написать самому.

In [317]:
import nltk
from nltk.collocations import *

In [318]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [319]:
finder2 = BigramCollocationFinder.from_documents(corpus)

In [320]:
finder3 = TrigramCollocationFinder.from_documents(corpus)

In [321]:
finder2.nbest(bigram_measures.likelihood_ratio, 20)

[('риа', 'новость'),
 ('итар', 'тасс'),
 ('владимир', 'путин'),
 ('сообщать', 'риа'),
 ('миллион', 'доллар'),
 ('правоохранительный', 'орган'),
 ('настоящее', 'время'),
 ('северный', 'кавказ'),
 ('1999', 'год'),
 ('пресс', 'служба'),
 ('2000', 'год'),
 ('санкт', 'петербург'),
 ('федеральный', 'сила'),
 ('эхо', 'москва'),
 ('уголовный', 'дело'),
 ('associated', 'press'),
 ('премьер', 'министр'),
 ('населить', 'пункт'),
 ('сей', 'пора'),
 ('взрывной', 'устройство')]

In [323]:
finder3.nbest(trigram_measures.pmi, 20)

[('110kb', '248kb', '403kb'),
 ('137000', 'впарный', '73750'),
 ('248kb', '403kb', '585kb'),
 ('3gb', 'hard', 'disk'),
 ('403kb', '585kb', '805kb'),
 ('585kb', '805kb', '1040kb'),
 ('885', 'telesonar', 'acoustic'),
 ('916', '7171', 'илинаправить'),
 ('abbey', 'gardy', 'squitieri'),
 ('abbyy', 'seico', 'epson'),
 ('advil', 'robitussin', 'chapstick'),
 ('anne', 'nouchi', 'laumounier'),
 ('anti', 'defamation', 'league'),
 ('atm', '885', 'telesonar'),
 ('bharatiya', 'janata', 'party'),
 ('bin', 'laden', 'kosovo'),
 ('brazil', 'p00', 'hackerz'),
 ('cherkassy', 'chernigov', 'chernovtsy'),
 ('chernigov', 'chernovtsy', 'crimea'),
 ('chernovtsy', 'crimea', 'dnepropetrovsk')]

Sklearn напрямую не предназначен для этого, но из него тоже можно вытаскивать устойчивые нграммы. Tfidf подходит как метрика.

In [327]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# по умолчанию векторайзер сам токенизирует, поэтому проще склеить токены через пробел
texts = [' '.join(sent) for sent in corpus]

In [350]:
tfidf = TfidfVectorizer(min_df=10, max_df=0.2, max_features=1000,
                       ngram_range=(2,2))

In [351]:
tfidf.fit(texts)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.2, max_features=1000, min_df=10,
        ngram_range=(2, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [352]:
# словарь со словами и индекасами
tfidf.vocabulary_

{'принимать участие': 655,
 'стать известно': 862,
 'станция мир': 861,
 'март 2000': 386,
 '2000 год': 14,
 'пресс конференция': 644,
 'мочь привести': 454,
 'космический станция': 351,
 'мощный взрыв': 456,
 'четыре человек': 968,
 'результат взрыв': 702,
 'агентство итар': 40,
 'итар тасс': 327,
 'ссылка источник': 856,
 'гувд москва': 207,
 'тяжёлый состояние': 911,
 'данные риа': 213,
 'риа новость': 710,
 'место происшествие': 404,
 'террористический акт': 892,
 'принять решение': 657,
 'миллиард доллар': 407,
 'премьер министр': 642,
 'вица премьер': 113,
 'премьер правительство': 643,
 'правительство россия': 609,
 'виктор христенко': 110,
 'сообщить интерфакс': 823,
 'последний месяц': 599,
 'составлять около': 841,
 'около миллиард': 527,
 'настоящее время': 467,
 'составить около': 838,
 'федеральный бюджет': 926,
 '1997 год': 10,
 'который находиться': 357,
 'миллион доллар': 410,
 'авторский право': 35,
 'тысяча человек': 908,
 'ближний день': 78,
 'международный валютный'

In [353]:
# массив с метриками, можно достать по индексу из словаря
tfidf.idf_

array([7.95733878, 7.71221632, 8.4053635 , 8.30217927, 8.26371298,
       8.02931228, 7.7796576 , 7.74536853, 7.44651316, 7.01364907,
       6.83132752, 6.27558021, 5.58631652, 8.20865321, 5.49112426,
       7.21012438, 8.24502085, 8.3421846 , 8.26371298, 8.44981526,
       8.02931228, 8.49633528, 6.41392197, 7.43821435, 8.05961763,
       8.42734241, 8.26371298, 6.97131471, 8.32198189, 8.24502085,
       8.10687051, 8.04435016, 8.07512181, 8.47280478, 8.47280478,
       8.28276118, 8.28276118, 7.69071012, 7.30675121, 7.97132502,
       8.10687051, 7.54265702, 8.01449719, 7.37419249, 8.47280478,
       8.20865321, 7.34365577, 8.3421846 , 8.36280389, 8.05961763,
       7.42182054, 8.26371298, 8.30217927, 8.3421846 , 8.36280389,
       7.87729607, 8.47280478, 8.19095363, 8.30217927, 8.07512181,
       8.36280389, 8.17356189, 7.21012438, 7.48041471, 7.04104805,
       8.44981526, 8.04435016, 8.12313103, 8.32198189, 8.32198189,
       8.44981526, 7.70140541, 7.38981781, 7.94354546, 7.98550

In [354]:
word2idf = []

for word, i in tfidf.vocabulary_.items():
    word2idf.append((tfidf.idf_[i], word))

In [356]:
sorted(word2idf, reverse=True)[:10]

[(8.938168032182691, 'рубль тонна'),
 (8.520432831482712, 'транспортный средство'),
 (8.520432831482712, 'дизельный топливо'),
 (8.520432831482712, 'военный служба'),
 (8.49633527990365, 'миллиард кубометр'),
 (8.49633527990365, 'который пройти'),
 (8.49633527990365, 'компьютерный безопасность'),
 (8.49633527990365, 'amazon com'),
 (8.472804782493458, 'центр общественный'),
 (8.472804782493458, 'суд признать')]

Преобразовать отдельный текст в список нграммов посложнее, но можно (правда порядок не получится сохранить). Будет полезно разобраться самому.